In [1]:
!pip install -U langchain-huggingface
!pip install \
    langchain_community \
    langchain_pinecone \
    langchain_openai \
    unstructured \
    langchain-text-splitters
!pip -q install langchain huggingface_hub transformers sentence_transformers
!pip install langchain_huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.8/355.8 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 1.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_

In [2]:
!pip install pinecone-client -q

In [3]:
!pip install --upgrade langchain openai -q
!pip install unstructured -q
!pip install unstructured[local-inference] -q
!pip install detectron2@git+https://github.com/facebookresearch/detectron2.git@v0.6#egg=detectron2 -q
!pip install transformers sentence_transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.5/112.5 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.4/466.4 kB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 64.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 87.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 64.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [25]:
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import DirectoryLoader #1
from langchain_text_splitters import RecursiveCharacterTextSplitter #2
import os
import glob
from sentence_transformers import SentenceTransformer
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.llms import HuggingFaceHub
from langchain.chains import RetrievalQA

In [5]:
from google.colab import userdata
os.environ['PINECONE_API_KEY']=userdata.get('PINECONE_API_KEY')
os.environ['HUGGINGFACE_API_TOKEN']=userdata.get('HUGGINGFACE_API_TOKEN')

In [6]:
directory = "/content/data"

def load_docs(directory):
    loader = DirectoryLoader(directory)
    documents = loader.load()
    return documents

documents = load_docs(directory)
len(documents)

4

In [7]:
def split_docs(documents, chunk_size=800, chunk_overlap=80):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    docs = text_splitter.split_documents(documents)
    return docs

docs = split_docs(documents)
len(docs)

682

In [8]:
docs[0].page_content

'Helping Patients Cope With A Traumatic Event\n\nWhat Is a Traumatic Event?\n\nAn event, or series of events, that causes moderate to severe stress reactions, is called a traumatic event. Traumatic events are characterized by a sense of horror, helplessness, serious injury, or the threat of serious injury or death. Traumatic events affect survivors, rescue workers, and friends and relatives of victims who have been directly involved. In addition to potentially affecting those who suffer injuries or loss. They may also affect people who have witnessed the event either firsthand or on television. Stress reactions immediately following a traumatic event are very common, however, most of the reactions will resolve within ten days.\n\nCommon Responses to a Traumatic Event'

In [9]:
from langchain.embeddings import SentenceTransformerEmbeddings
embeddings = SentenceTransformerEmbeddings(model_name="neuml/pubmedbert-base-embeddings")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/6.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/706k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
query_result = embeddings.embed_query("I feel so overwhelmed and alone. Can you give me some support?")
len(query_result)

768

In [21]:
# from langchain.vectorstores import Pinecone
# index_name = "langchain-sdemo"

# index = Pinecone.from_documents(docs, embeddings, index_name=index_name)
# def get_similar_docs(query, k=2, score=False):
#     if score:
#         similar_docs = index.similarity_search_with_score(query, k=k)
#     else:
#         similar_docs = index.similarity_search(query, k=k)
#     return similar_docs

# query = "how to be happy?"
# similar_docs = get_similar_docs(query)
# similar_docs

[Document(metadata={'source': '/content/data/02529b9b-a34b-4fe9-bb0e-75528342d1a9.pdf'}, page_content='happening right now?’ ‘Are you able to hear me?’\n\nii. Reassure: Tell them they are safe – ‘You are (NAME), you are in the dental surgery with (PRACTITIONER NAME/anyone else who is present).’ ‘You are not in danger.’\n\niii. Normalise: Let the client understand that dissociation is a\n\nnormal bodily process that evolved to keep us safe, that even if it is scary, it is not dangerous and it is their body doing its best to protect them. If you feel able, you could explain a bit about the ‘freeze’ response using the ‘Window of tolerance’ diagrams.\n\niv. Help the client to self-soothe:'),
 Document(metadata={'source': '/content/data/2d39e446-a0bb-4d37-ab42-867b31860c17.pdf'}, page_content='Fourth, judges state that they need help in learning how to listen to children tell about the horrors they have experienced at the hands of their parents and then respond to the child in a helpful, ap

In [15]:
from langchain_pinecone import PineconeVectorStore
index_name = "langchain-fdemo"
vectorstore = PineconeVectorStore.from_documents(docs, embeddings, index_name=index_name)

# query = "how to be happy?"
# similar_docs = vectorstore.similarity_search(query)
# similar_docs

[Document(metadata={'source': '/content/data/02529b9b-a34b-4fe9-bb0e-75528342d1a9.pdf'}, page_content='Appear compliant/submissive\n\nAppear not to feel pain\n\n²© British Dental Association 2023. ª© The Author(s) under exclusive licence to British Dental Association 2023.\n\n\uf038\n\n491\n\n\uf037\n\nUpFrONT\n\nº Ask the client to recall where they are in the present – the date, time, location, and who is with them. Alternatively, to help reorient them in the present you could ask them to list, either out loud or in their head, things that they can see noticing the different colours and textures, in their surroundings. v. Make a contract with them on what they would like you to do if this happens again: Ask them if there’s anything they would like you to do differently if they dissociate again.\n\nSee Table 3 for some tips on how to sensitively respond to\n\ndisclosures.19'),
 Document(metadata={'source': '/content/data/02529b9b-a34b-4fe9-bb0e-75528342d1a9.pdf'}, page_content='Pass t

In [22]:
query = "how to be happy?"
similar_docs = vectorstore.similarity_search(query)
similar_docs

[Document(metadata={'source': '/content/data/02529b9b-a34b-4fe9-bb0e-75528342d1a9.pdf'}, page_content='happening right now?’ ‘Are you able to hear me?’\n\nii. Reassure: Tell them they are safe – ‘You are (NAME), you are in the dental surgery with (PRACTITIONER NAME/anyone else who is present).’ ‘You are not in danger.’\n\niii. Normalise: Let the client understand that dissociation is a\n\nnormal bodily process that evolved to keep us safe, that even if it is scary, it is not dangerous and it is their body doing its best to protect them. If you feel able, you could explain a bit about the ‘freeze’ response using the ‘Window of tolerance’ diagrams.\n\niv. Help the client to self-soothe:'),
 Document(metadata={'source': '/content/data/2d39e446-a0bb-4d37-ab42-867b31860c17.pdf'}, page_content='Fourth, judges state that they need help in learning how to listen to children tell about the horrors they have experienced at the hands of their parents and then respond to the child in a helpful, ap

In [23]:
from langchain_community.llms import HuggingFaceHub
from langchain.chains import RetrievalQA

# Define the repo ID and connect to Mixtral model on Huggingface
repo_id = "tiiuae/falcon-7b-instruct"
llm = HuggingFaceHub(
    huggingfacehub_api_token=os.getenv("HUGGINGFACE_API_TOKEN"),
    repo_id=repo_id,
    model_kwargs={"temperature": 0.7, "max_new_tokens": 500},
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

qa.invoke(query)

{'query': 'how to be happy?',
 'result': "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nhappening right now?’ ‘Are you able to hear me?’\n\nii. Reassure: Tell them they are safe – ‘You are (NAME), you are in the dental surgery with (PRACTITIONER NAME/anyone else who is present).’ ‘You are not in danger.’\n\niii. Normalise: Let the client understand that dissociation is a\n\nnormal bodily process that evolved to keep us safe, that even if it is scary, it is not dangerous and it is their body doing its best to protect them. If you feel able, you could explain a bit about the ‘freeze’ response using the ‘Window of tolerance’ diagrams.\n\niv. Help the client to self-soothe:\n\nFourth, judges state that they need help in learning how to listen to children tell about the horrors they have experienced at the hands of their parents and then respond to the child in a helpful, 

In [18]:
# Integrate the PineconeVectorStore into the context
context = vectorstore.as_retriever()

In [16]:
from langchain_community.llms import HuggingFaceHub

# Define the repo ID and connect to Mixtral model on Huggingface
repo_id = "tiiuae/falcon-7b-instruct"
llm = HuggingFaceHub(
    huggingfacehub_api_token=os.getenv("HUGGINGFACE_API_TOKEN"),
    repo_id=repo_id,
    model_kwargs={"temperature": 0.7, "max_new_tokens": 500},
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEndpoint`.
  warn_deprecated(


In [19]:
from langchain_core.prompts import PromptTemplate

# Prompt engineering
template = """
        You are a mental health adviser. These Human will ask you a questions about their life.
        Use following piece of context to answer the question.
        If you don't know the answer, just say you don't know.
        Keep the answer within 2 sentences and concise.

        Context: {context}
        Question: {question}
        Answer:
"""
prompt = PromptTemplate(
  template=template, input_variables=["context", "question"]
)

In [20]:
from langchain_pinecone import PineconeVectorStore
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

rag_chain = (
    {
        "context": context,  # Use the 'context' variable created above
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)